https://www.gymlibrary.ml/

## GYM官方GitHub地址

In [1]:
!pip install gym
!pip install pygame

## 使用

In [2]:
import gym
env = gym.make("CartPole-v0")
env.action_space.seed(42)

observation, info = env.reset(seed=42, return_info=True)

for _ in range(1000):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())

    if done:
        observation, info = env.reset(return_info=True)

env.close()

E:\install\anaconda\envs\algo_note\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [3]:
import gym
env = gym.make("MountainCar-v0")

# plt.imshow(env.render('rgb_array'))
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

Observation space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action space: Discrete(3)


## Space


### Discrete



描述一个离散空间，其中{0，1，…，n-1}是我们的观察或操作可以采取的可能值。可以使用可选参数将值移动到{a，a+1，…，a+n-1}。

查看一下倒立摆的`action_space`

In [4]:
env = gym.make("CartPole-v0")
env.action_space

Discrete(2)

其源码定义如下:

```python
self.action_space = spaces.Discrete(2)
```

我们再查看一下Discrete这个类，其源码地址为:

Space的官方源码地址为: https://github.com/openai/gym/tree/master/gym/spaces。

可以看到其有两种创建方式:

```python
>>> Discrete(2)            # {0, 1}
>>> Discrete(3, start=-1)  # {-1, 0, 1}
```

其函数说明里面也说了就是创建离散的动作的。

In [5]:
from gym import spaces

In [6]:
spaces.Discrete(2, start=-1)

Discrete(2, start=-1)

### Box

描述了一个n维连续空间。是一个有界空间，我们可以定义上限和下限，来描述我们的观测值的有效值。

查看一下"CartPole-v0"的Box定义:

In [7]:
env = gym.make("CartPole-v0")
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

在其源码中observation_space的定义如下:

```python
high = np.array(
    [
        self.x_threshold * 2,
        np.finfo(np.float32).max,
        self.theta_threshold_radians * 2,
        np.finfo(np.float32).max,
    ],
    dtype=np.float32,
)
self.observation_space = spaces.Box(-high, high, dtype=np.float32)
```

### Dict

用于表示简单`spaces`的字典。

官方源码地址为:https://github1s.com/openai/gym/blob/master/gym/spaces/dict.py

In [8]:
from gym.spaces import Dict
observation_space = Dict({"position": spaces.Discrete(2), "velocity": spaces.Discrete(3)})
observation_space.sample()

OrderedDict([('position', 0), ('velocity', 2)])

### Tuple

表示简单spaces的元祖

官方源码地址为:

In [9]:
from gym.spaces import Tuple
observation_space = spaces.Tuple((spaces.Discrete(2), spaces.Discrete(3)))
observation_space.sample()

(1, 0)

### MultiBinary

表示一个n维的二进制空间

In [10]:
from gym.spaces import MultiBinary
observation_space = MultiBinary(5)
observation_space.sample()

array([1, 0, 1, 0, 1], dtype=int8)

In [11]:
from gym.spaces import MultiBinary
observation_space = MultiBinary((3, 2))
observation_space.sample()

array([[0, 1],
       [1, 1],
       [0, 1]], dtype=int8)

### MultiDiscrete

由一系列离散的动作空间组成，每个元素中有不同数量的动作。

In [12]:
from gym.spaces import MultiDiscrete

observation_space = MultiDiscrete([5, 2, 2])
observation_space.sample()

array([0, 0, 0], dtype=int64)

## Wrappers

Wrappers是一种修改现有环境的便捷方式，无需直接修改底层代码。使用Wrappers将允许您避免大量样板代码，并使您的环境更加模块化。Wrappers也可以链接起来，以组合其效果。大多数环境都是通过gym生成的。默认情况下，make已被包装。

为了包装环境，必须首先初始化基本环境。然后，可以将此环境以及（可能是可选的）参数传递给包装器的构造函数：

### RescaleAction

`RescaleAction`能够将连续动作空间中的动作，缩放到`[min_action, max_action]`这个范围。


In [13]:
import gym
from gym.wrappers import RescaleAction

base_env = gym.make('BipedalWalker-v3')

In [14]:
base_env.action_space

Box(-1.0, 1.0, (4,), float32)

In [15]:
wrapper_env = RescaleAction(base_env, min_action=0, max_action=1)
wrapper_env.action_space

Box(0.0, 1.0, (4,), float32)

除此之外，还有对观测和奖励的wrappers。

## 向量化API

`Vectorized Environments`是一次运行多个环境，采用并行或者串行的方式。对应的动作和奖励也是一个`batch`的。

In [16]:
envs = gym.vector.make('CartPole-v1', num_envs=3)
envs.reset()
observations, rewards, dones, infos = envs.step([0, 1, 0])

In [17]:
observations

array([[ 0.04450928, -0.24112365, -0.02487868,  0.25696984],
       [ 0.04389759,  0.22447157,  0.01153603, -0.30892634],
       [ 0.04329293, -0.16258499,  0.0089608 ,  0.30264556]],
      dtype=float32)

GYM提供了两种向量化环境的方式:

1. gym.vector.SyncVectorEnv  （序列化执行）
2. gym.vector.AsyncVectorEnv   （并行执行）

In [18]:
envs = gym.vector.AsyncVectorEnv([
    lambda : gym.make('Pendulum-v1', g=9.81),
    lambda : gym.make('Pendulum-v1', g=1.62)
])
observations = envs.reset()
print(observations)

[[-0.33599427  0.941864   -0.2547292 ]
 [ 0.85120684 -0.52483034  0.73773336]]
